In [8]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd


In [9]:
def get_listings(search_page):
    answer = requests.get(search_page, timeout=500)
    content = answer.content
    soup = BeautifulSoup(content, "html.parser")
    listings = soup.find_all("div", "cm4lcvy")
    return listings


In [10]:
def extract_element(listing_html, params):
    # 1. Find the right tag
    if "class" in params:
        elements_found = listing_html.find_all(params["tag"], params["class"])
    else:
        elements_found = listing_html.find_all(params["tag"])

    # 2. Extract the right element
    tag_order = params.get("order", 0)
    element = elements_found[tag_order]

    # 3. Get text
    if "get" in params:
        output = element.get(params["get"])
    else:
        output = element.get_text()

    return output


In [11]:
def extract_page_features(soup, rules):
    features_dict = {}
    for feature in rules:
        try:
            features_dict[feature] = extract_element(soup, rules[feature])
        except:
            features_dict[feature] = "empty"

    return features_dict


In [12]:
def build_urls(main_url, listings_per_page=20, pages_per_location=15):
    url_list = []
    for i in range(pages_per_location):
        offset = listings_per_page * i
        url_pagination = main_url + f"&items_offset={offset}"
        url_list.append(url_pagination)

    return url_list


In [13]:
def process_search_pages(url_list):
    features_list = []
    for page in url_list:
        listings = get_listings(page)
        for listing in listings:
            features = extract_page_features(listing, RULES_SEARCH_PAGE)
            features_list.append(features)

    return features_list


In [14]:
airbnb_market_listings = {
    "Joshua Tree CA": "https://www.airbnb.com/s/Joshua-Tree--California--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&query=Joshua%20Tree%2C%20California%2C%20United%20States&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&place_id=ChIJAVDJ-0XQ2oARw2sUAFr-5-Q&adults=1",
    "Austin TX": "https://www.airbnb.com/s/austin-tx/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=user_map_move&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Austin%2C%20Texas%2C%20United%20States&ne_lat=30.667059027669115&ne_lng=-96.80834546067751&sw_lat=29.994395524834584&sw_lng=-98.5654713151697&zoom=11&search_by_map=true",
    "Pueblo CO": "https://www.airbnb.com/s/Pueblo--Colorado--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Pueblo%2C%20Colorado%2C%20United%20States&place_id=ChIJZbEbB_OiE4cRLrjHaKNrLag",
    "Vail CO": "https://www.airbnb.com/s/Vail--Colorado--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Vail%2C%20Colorado%2C%20United%20States&place_id=ChIJB89dUQVwaocRxKOafh_AzMk",
    "Great Capacon WV": "https://www.airbnb.com/s/Great-Cacapon--West-Virginia--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Great%20Cacapon%2C%20West%20Virginia%2C%20United%20States&place_id=ChIJawt3t0wTyokROMLwDAHkLH0",
    "Shenandoah VA": "https://www.airbnb.com/s/Shenandoah--Virginia--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Shenandoah%2C%20Virginia%2C%20United%20States&place_id=ChIJFZKaBwxftIkREgOL6XGvDsI",
    "Asheville NC": "https://www.airbnb.com/s/Asheville--North-Carolina--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Asheville%2C%20North%20Carolina%2C%20United%20States&place_id=ChIJCW8PPKmMWYgRXTo0BsEx75Q",
    "Windham NY": "https://www.airbnb.com/s/Windham--New-York--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Windham%2C%20New%20York%2C%20United%20States&place_id=ChIJuUVJxhdL3IkRLjRgnp2qi4o",
    "Eureka CA": "https://www.airbnb.com/s/Eureka--California--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Eureka%2C%20California%2C%20United%20States&place_id=ChIJp07_5oP_01QRlQYpuGYHhio",
    "Conway NH": "https://www.airbnb.com/s/Conway--New-Hampshire--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Conway%2C%20New%20Hampshire%2C%20United%20States&place_id=ChIJRYAKt3QIs0wRlihEBCRg0NA",
    "Kennebunkport ME": "https://www.airbnb.com/s/Kennebunkport--Maine--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Kennebunkport%2C%20Maine%2C%20United%20States&place_id=ChIJ6YMt4z2pskwRHz_HYk7Wh7w",
    "Missoula MT": "https://www.airbnb.com/s/Missoula--Montana--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Missoula%2C%20Montana%2C%20United%20States&place_id=ChIJy2fzUCrMXVMRLoCUyncS4-k",
    "Dolores CO": "https://www.airbnb.com/s/Dolores--Colorado--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Dolores%2C%20Colorado%2C%20United%20States&place_id=ChIJA93DHiAwOYcRfCfkpufte4E",
    "Ithica NY": "https://www.airbnb.com/s/Ithaca--New-York--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Ithaca%2C%20New%20York%2C%20United%20States&place_id=ChIJ94iv4IKB0IkRdM5upWgn5Qo",
    "Portland ME": "https://www.airbnb.com/s/Portland--Maine--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Portland%2C%20Maine%2C%20United%20States&place_id=ChIJLe6wqnKcskwRKfpyM7W2nX4&federated_search_session_id=0561964d-ba67-4290-b2dd-17800af2deb7&pagination_search=true&items_offset=20&section_offset=3",
    "Eugene OR": "https://www.airbnb.com/s/Eugene--Oregon--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Eugene%2C%20Oregon%2C%20United%20States&place_id=ChIJGRlQrLAZwVQRTYlDSolh7Fc&federated_search_session_id=8bfaf0f5-093d-4d99-b495-424a85345953&pagination_search=true",
    "Knoxville TN": "https://www.airbnb.com/s/Knoxville--Tennessee--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Knoxville%2C%20Tennessee%2C%20United%20States&place_id=ChIJqULORiIWXIgRxTT1xNqS6ns&federated_search_session_id=82abab2f-9a64-480f-9172-441be3744628&pagination_search=true&items_offset=20&section_offset=3",
    "Moab UT": "https://www.airbnb.com/s/Moab--Utah--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Moab%2C%20Utah%2C%20United%20States&place_id=ChIJjS24muXhR4cRLcTV8XqxMgs",
    "Osage Beach MO": "https://www.airbnb.com/s/Osage-Beach--Missouri--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Osage%20Beach%2C%20Missouri%2C%20United%20States&place_id=ChIJcSh0LiTFxIcRRxVddK0mdWs",
    "Harpers Ferry WV": "https://www.airbnb.com/s/Harpers-Ferry--West-Virginia--United-States/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&l2_property_type_ids%5B%5D=5&l2_property_type_ids%5B%5D=7&l2_property_type_ids%5B%5D=10&click_referer=t%3ASEE_ALL%7Csid%3A2124eabb-77e7-4299-89a1-c7b26157f156%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&last_search_session_id=2124eabb-77e7-4299-89a1-c7b26157f156&tab_id=home_tab&adults=1&query=Harpers%20Ferry%2C%20West%20Virginia%2C%20United%20States&place_id=ChIJTZXx7A0DtokRHs20WKeaW9Y",
    "Texas Hill Country, TX" : "https://www.airbnb.com/s/Hill-Country--San-Antonio--Texas--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&query=Hill%20Country%2C%20San%20Antonio%2C%20TX%2078247%2C%20USA&place_id=ChIJ8YrLI2qMXIYRPEiAmVKbuyk&source=structured_search_input_header&search_type=user_map_move&ne_lat=32.1338938804884&ne_lng=-94.54842104585725&sw_lat=27.298714191615648&sw_lng=-100.52464022310335&zoom=7&search_by_map=true&l2_property_type_ids%5B%5D=5",
    "West Texas, TX":"https://www.airbnb.com/s/Hill-Country--San-Antonio--Texas--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&query=Hill%20Country%2C%20San%20Antonio%2C%20TX%2078247%2C%20USA&place_id=ChIJ8YrLI2qMXIYRPEiAmVKbuyk&source=structured_search_input_header&search_type=user_map_move&ne_lat=32.082017882674485&ne_lng=-100.40991399657986&sw_lat=28.95951382928442&sw_lng=-104.51330755126736&zoom=8&search_by_map=true&l2_property_type_ids%5B%5D=5",
}


In [15]:
RULES_SEARCH_PAGE = {
    "url": {"tag": "a", "get": "href"},
    "name": {"tag": "div", "class": "c1bx80b8"},
    "header": {"tag": "div", "class": "mj1p6c8"},
    "rooms": {"tag": "div", "class": "i1wgresd", "order": 0},
    #     'facilities': {'tag': 'div', 'class': '_kqh46o', 'order': 1},
    #     'badge': {'tag': 'div', 'class': '_17bkx6k'},
    #     'rating_n_reviews': {'tag': 'span', 'class': '_18khxk1'},
    "price": {"tag": "span", "class": "_tyxjp1"},
    #     'superhost': {'tag': 'div', 'class': '_ufoy4t'},
}


In [16]:
dfs = []
for city, url in tqdm(airbnb_market_listings.items()):
    url_list = build_urls(url)
    base_features = process_search_pages(url_list)
    df = pd.DataFrame(base_features)
    df["city"] = city
    df["original_url"] = url
    dfs.append(df)


  0%|          | 0/22 [00:00<?, ?it/s]

In [17]:
df_out = pd.concat(dfs)


In [18]:
data_cols = ["guests", "bedrooms", "beds", "baths"]

df_out[data_cols] = df_out["rooms"].str.split(" · ", expand=True)

df_out["url"] = "https://www.airbnb.com" + df_out["url"]


In [19]:
df_out.head()


,url,name,header,rooms,price,city,original_url,guests,bedrooms,beds,baths
0,https://www.airbnb.com/rooms/45191987?adults=1...,Airstream Oasis in Joshua Tree - A/C and Hot Tub.,Camper/RV in Joshua Tree,4 guests · 1 bedroom · 1 bed · 1 bath,$254,Joshua Tree CA,https://www.airbnb.com/s/Joshua-Tree--Californ...,4 guests,1 bedroom,1 bed,1 bath
1,https://www.airbnb.com/rooms/44203588?adults=1...,Modern Airstream w/ outdoor movies & wood hot tub,Camper/RV in Joshua Tree,2 guests · 1 bedroom · 1 bed · 1 bath,$285,Joshua Tree CA,https://www.airbnb.com/s/Joshua-Tree--Californ...,2 guests,1 bedroom,1 bed,1 bath
2,https://www.airbnb.com/rooms/21518766?adults=1...,CHIQUITA ON FLAMINGO in Joshua Tree,Camper/RV in Joshua Tree,4 guests · 1 bedroom · 1 bed · 1 bath,$146,Joshua Tree CA,https://www.airbnb.com/s/Joshua-Tree--Californ...,4 guests,1 bedroom,1 bed,1 bath
3,https://www.airbnb.com/rooms/55014606519546052...,The Libra Tent at Purple Peaks,Tent in Twentynine Palms,2 guests · 1 bedroom · 1 bed · 1 bath,$75,Joshua Tree CA,https://www.airbnb.com/s/Joshua-Tree--Californ...,2 guests,1 bedroom,1 bed,1 bath
4,https://www.airbnb.com/rooms/30824149?adults=1...,Luxury Desert Camping for 2,Camper/RV in Yucca Valley,2 guests · 1 bedroom · 1 bed · 1 bath,$167,Joshua Tree CA,https://www.airbnb.com/s/Joshua-Tree--Californ...,2 guests,1 bedroom,1 bed,1 bath


In [20]:
for i in data_cols + ["price"]:
    df_out[i] = df_out[i].replace("Studio", "0")
    df_out[i] = df_out[i].astype(str).str.extract("(\d+)")
    df_out[i] = pd.to_numeric(df_out[i], errors="coerce")


In [21]:
df_out


,url,name,header,rooms,price,city,original_url,guests,bedrooms,beds,baths
0,https://www.airbnb.com/rooms/45191987?adults=1...,Airstream Oasis in Joshua Tree - A/C and Hot Tub.,Camper/RV in Joshua Tree,4 guests · 1 bedroom · 1 bed · 1 bath,254,Joshua Tree CA,https://www.airbnb.com/s/Joshua-Tree--Californ...,4,1,1.0,1.0
1,https://www.airbnb.com/rooms/44203588?adults=1...,Modern Airstream w/ outdoor movies & wood hot tub,Camper/RV in Joshua Tree,2 guests · 1 bedroom · 1 bed · 1 bath,285,Joshua Tree CA,https://www.airbnb.com/s/Joshua-Tree--Californ...,2,1,1.0,1.0
2,https://www.airbnb.com/rooms/21518766?adults=1...,CHIQUITA ON FLAMINGO in Joshua Tree,Camper/RV in Joshua Tree,4 guests · 1 bedroom · 1 bed · 1 bath,146,Joshua Tree CA,https://www.airbnb.com/s/Joshua-Tree--Californ...,4,1,1.0,1.0
3,https://www.airbnb.com/rooms/55014606519546052...,The Libra Tent at Purple Peaks,Tent in Twentynine Palms,2 guests · 1 bedroom · 1 bed · 1 bath,75,Joshua Tree CA,https://www.airbnb.com/s/Joshua-Tree--Californ...,2,1,1.0,1.0
4,https://www.airbnb.com/rooms/30824149?adults=1...,Luxury Desert Camping for 2,Camper/RV in Yucca Valley,2 guests · 1 bedroom · 1 bed · 1 bath,167,Joshua Tree CA,https://www.airbnb.com/s/Joshua-Tree--Californ...,2,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
203,https://www.airbnb.com/rooms/46509853?adults=1...,BIKERS HARBOR #7,Campsite in Terlingua,9 guests · 1 bedroom · Half-bath,20,"West Texas, TX",https://www.airbnb.com/s/Hill-Country--San-Ant...,9,1,NaN,NaN
204,https://www.airbnb.com/rooms/47178688?adults=1...,RV Site #6 (RV NOT INCLUDED) TinValleyRetroRen...,Camper/RV in Terlingua,4 guests · 1 bedroom · 1 bed · 1 bath,30,"West Texas, TX",https://www.airbnb.com/s/Hill-Country--San-Ant...,4,1,1.0,1.0
205,https://www.airbnb.com/rooms/47177602?adults=1...,RV Site #5 (RV NOT INCLUDED) TinValleyRetroRen...,Camper/RV in Terlingua,1 guest · 1 bedroom · 1 bath,30,"West Texas, TX",https://www.airbnb.com/s/Hill-Country--San-Ant...,1,1,1.0,NaN
206,https://www.airbnb.com/rooms/43870950?adults=1...,Quite RV park in close to city but in country,Campsite in Midland,4 guests · Studio · 2 beds,25,"West Texas, TX",https://www.airbnb.com/s/Hill-Country--San-Ant...,4,0,2.0,NaN


In [22]:
df_out.to_csv("airbnb-glamping.csv")


In [23]:
# try for one page
#base_features = process_search_pages(url_list[4:5])
